# LGBM prediction 1차
- feature :
    - EDA에서 확인한 모든 feature 공통 feature로 추가 (시간 관련, 기온 관련)
    - 일사, 일조, 강수량 feature 일괄 제거
    - 유형별 작업 안함
- modeling :
    - LGBM
- 이상치:
    - hourly 이동평균선 기준 하한선 이상치는 하한선 한계값으로 대체함


- 결과 (평균)
 - 건물기타	4.977648417
 - 공공	5.065291864
 - 대학교	3.694792189
 - 데이터센터	0.2794135276



---



# building 1-36 데이터 준비

- # 건물기타 15개, 공공 8개, 대학교 8개, 데이터센터 5개

In [2297]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sktime

In [2298]:
from lightgbm import LGBMRegressor, early_stopping
from xgboost import XGBRegressor, callback

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder

from sktime.forecasting.model_selection import temporal_train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer


In [2299]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/데이콘 캐글 컴페티션/DACON_23.07_PowerConsumption/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [2300]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
building = pd.read_csv(f"{DATA_PATH}building_info.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [2301]:
def _smape(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    output = np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100
    return output
smape = make_scorer(_smape, greater_is_better=False)

In [2302]:
building['건물유형'].value_counts()

건물기타       15
공공          8
대학교         8
백화점및아울렛     8
병원          8
상용          8
아파트         8
연구소         8
지식산업센터      8
할인마트        8
호텔및리조트      8
데이터센터       5
Name: 건물유형, dtype: int64

In [2303]:
train_ft = pd.merge(train,building,on='건물번호',how='left')
train_ft

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,건물기타,110634.00,39570.00,-,-,-
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,건물기타,110634.00,39570.00,-,-,-
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,건물기타,110634.00,39570.00,-,-,-
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76,건물기타,110634.00,39570.00,-,-,-
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40,건물기타,110634.00,39570.00,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04,호텔및리조트,57497.84,40035.23,-,-,-
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96,호텔및리조트,57497.84,40035.23,-,-,-
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12,호텔및리조트,57497.84,40035.23,-,-,-
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08,호텔및리조트,57497.84,40035.23,-,-,-


In [2304]:
test_ft = pd.merge(test,building,on='건물번호',how='left')
test_ft

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72,건물기타,110634.00,39570.00,-,-,-
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72,건물기타,110634.00,39570.00,-,-,-
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75,건물기타,110634.00,39570.00,-,-,-
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78,건물기타,110634.00,39570.00,-,-,-
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77,건물기타,110634.00,39570.00,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100_20220831 19,100,20220831 19,22.5,0.0,0.9,84,호텔및리조트,57497.84,40035.23,-,-,-
16796,100_20220831 20,100,20220831 20,20.7,0.0,0.4,95,호텔및리조트,57497.84,40035.23,-,-,-
16797,100_20220831 21,100,20220831 21,20.2,0.0,0.4,98,호텔및리조트,57497.84,40035.23,-,-,-
16798,100_20220831 22,100,20220831 22,20.1,0.0,1.1,97,호텔및리조트,57497.84,40035.23,-,-,-


In [2305]:
train_ft.columns

Index(['num_date_time', '건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형', '연면적(m2)', '냉방면적(m2)',
       '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'],
      dtype='object')

In [2306]:
test_ft.columns

Index(['num_date_time', '건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)',
       'PCS용량(kW)'],
      dtype='object')

- object 칼럼 처리:  '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'

In [2307]:
train_ft['태양광용량(kW)'] = train_ft['태양광용량(kW)'].replace('-', np.nan).astype(float)
train_ft['ESS저장용량(kWh)'] = train_ft['ESS저장용량(kWh)'].replace('-', np.nan).astype(float)
train_ft['PCS용량(kW)'] = train_ft['PCS용량(kW)'].replace('-', np.nan).astype(float)

In [2308]:
test_ft['태양광용량(kW)'] = test_ft['태양광용량(kW)'].replace('-', np.nan).astype(float)
test_ft['ESS저장용량(kWh)'] = test_ft['ESS저장용량(kWh)'].replace('-', np.nan).astype(float)
test_ft['PCS용량(kW)'] = test_ft['PCS용량(kW)'].replace('-', np.nan).astype(float)

In [2309]:
train_ft

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,건물기타,110634.00,39570.00,NaN,NaN,NaN
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,건물기타,110634.00,39570.00,NaN,NaN,NaN
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,건물기타,110634.00,39570.00,NaN,NaN,NaN
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76,건물기타,110634.00,39570.00,NaN,NaN,NaN
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40,건물기타,110634.00,39570.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04,호텔및리조트,57497.84,40035.23,NaN,NaN,NaN
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96,호텔및리조트,57497.84,40035.23,NaN,NaN,NaN
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12,호텔및리조트,57497.84,40035.23,NaN,NaN,NaN
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08,호텔및리조트,57497.84,40035.23,NaN,NaN,NaN


In [2310]:
train_ft.isnull().sum()

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
일조(hr)            75182
일사(MJ/m2)         87913
전력소비량(kWh)            0
건물유형                  0
연면적(m2)               0
냉방면적(m2)              0
태양광용량(kW)        130560
ESS저장용량(kWh)     193800
PCS용량(kW)        193800
dtype: int64

- object 칼럼 처리 : 'num_date_time'제거 , '일시' to datetime

In [2311]:
train_ft['일시'] = pd.to_datetime(train_ft['일시'])

# 월, 일, 시간 추출하여 새로운 칼럼 추가
train_ft['월'] = train_ft['일시'].dt.month
train_ft['일'] = train_ft['일시'].dt.day
train_ft['시간'] = train_ft['일시'].dt.hour

# 요일 추출하여 새로운 칼럼 추가 (월요일: 0, 일요일: 6)
train_ft['요일'] = train_ft['일시'].dt.weekday

#일자번호 추가
train_ft['일자번호'] = train_ft['일시'].dt.dayofyear

#주간번호 추가
# train_ft['주간번호'] = train_ft['일시'].dt.isocalendar().week


In [2312]:
test_ft['일시'] = pd.to_datetime(train_ft['일시'])

# 월, 일, 시간 추출하여 새로운 칼럼 추가
test_ft['월'] = test_ft['일시'].dt.month
test_ft['일'] = test_ft['일시'].dt.day
test_ft['시간'] = test_ft['일시'].dt.hour

# 요일 추출하여 새로운 칼럼 추가 (월요일: 0, 일요일: 6)
test_ft['요일'] = test_ft['일시'].dt.weekday

#일자번호 추가
test_ft['일자번호'] = test_ft['일시'].dt.dayofyear

#주간번호 추가
# test_ft['주간번호'] = test_ft['일시'].dt.isocalendar().week


In [2313]:
train_ft = train_ft.drop('num_date_time', axis=1)
test_ft = test_ft.drop('num_date_time', axis=1)


In [2314]:
train_ft.fillna(0)

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),월,일,시간,요일,일자번호
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,건물기타,110634.00,39570.00,0.0,0.0,0.0,6,1,0,2,152
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,건물기타,110634.00,39570.00,0.0,0.0,0.0,6,1,1,2,152
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,건물기타,110634.00,39570.00,0.0,0.0,0.0,6,1,2,2,152
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,건물기타,110634.00,39570.00,0.0,0.0,0.0,6,1,3,2,152
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,건물기타,110634.00,39570.00,0.0,0.0,0.0,6,1,4,2,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0,8,24,19,2,236
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0,8,24,20,2,236
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0,8,24,21,2,236
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0,8,24,22,2,236


- 칼럼추가

- 데이터 칼럼 영문화

In [2315]:
train_ft.columns

Index(['건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전력소비량(kWh)', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)',
       'ESS저장용량(kWh)', 'PCS용량(kW)', '월', '일', '시간', '요일', '일자번호'],
      dtype='object')

In [2316]:
test_ft.columns

Index(['건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '건물유형', '연면적(m2)',
       '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '월', '일', '시간',
       '요일', '일자번호'],
      dtype='object')

In [2317]:
# 변수 영문명으로 변경
train_eng_cols = ['building_num', 'date_time', 'temp', 'precip', 'wind_ms', 'humidity', 'sunshine',
                  'solar_rad', 'energy_consum', 'building_type', 'floor_area', 'cooling_area', 'solar_capa',
                  'ess_capa','pcs_capa','month','day','hour','day_of_week','day_of_year']

train_ft.columns = train_eng_cols
# test_df.columns = test_eng_cols

In [2318]:
test_eng_cols = ['building_num', 'date_time', 'temp', 'precip', 'wind_ms', 'humidity','building_type', 'floor_area',
                 'cooling_area', 'solar_capa', 'ess_capa','pcs_capa','month','day','hour',
                 'day_of_week','day_of_year']

test_ft.columns = test_eng_cols

In [2319]:
train_ft.columns

Index(['building_num', 'date_time', 'temp', 'precip', 'wind_ms', 'humidity',
       'sunshine', 'solar_rad', 'energy_consum', 'building_type', 'floor_area',
       'cooling_area', 'solar_capa', 'ess_capa', 'pcs_capa', 'month', 'day',
       'hour', 'day_of_week', 'day_of_year'],
      dtype='object')

- 피처 추가

In [2320]:
# 시간에 대한 cos, sin 피처 추가
train_ft['cos_hour'] = np.cos(2 * np.pi * train_ft['hour'] / 24)  # 하루 기준으로 cos 값 추가
train_ft['sin_hour'] = np.sin(2 * np.pi * train_ft['hour'] / 24)  # 하루 기준으로 sin 값 추가

# 일에 대한 주기성을 감지하기 위한 피처 추가
train_ft['day_of_year'] = train_ft['month'] * 30 + train_ft['day']  # 간단한 근사 값
train_ft['cos_day_of_year'] = np.cos(2 * np.pi * train_ft['day_of_year'] / 365)
train_ft['sin_day_of_year'] = np.sin(2 * np.pi * train_ft['day_of_year'] / 365)

# 요일에 대한 sin, cos 피처 추가
train_ft['cos_weekday'] = np.cos(2 * np.pi * train_ft['day_of_week'] / 7)  # 7일 기준으로 cos 값 추가
train_ft['sin_weekday'] = np.sin(2 * np.pi * train_ft['day_of_week'] / 7)  # 7일 기준으로 sin 값 추가

# 주차별 cos, sin 피처 추가
train_ft['cos_week'] = np.cos(2 * np.pi * train_ft['day_of_year'] / 52)  # 52주 기준으로 cos 값 추가
train_ft['sin_week'] = np.sin(2 * np.pi * train_ft['day_of_year'] / 52)  # 52주 기준으로 sin 값 추가

# 월에 대한 cos, sin 피처 추가
train_ft['cos_month'] = np.cos(2 * np.pi * train_ft['month'] / 12)  # 12달 기준으로 cos 값 추가
train_ft['sin_month'] = np.sin(2 * np.pi * train_ft['month'] / 12)  # 12달 기준으로 sin 값 추가


In [2321]:
# 시간에 대한 cos, sin 피처 추가
test_ft['cos_hour'] = np.cos(2 * np.pi * test_ft['hour'] / 24)  # 하루 기준으로 cos 값 추가
test_ft['sin_hour'] = np.sin(2 * np.pi * test_ft['hour'] / 24)  # 하루 기준으로 sin 값 추가

# 일에 대한 주기성을 감지하기 위한 피처 추가
test_ft['day_of_year'] = test_ft['month'] * 30 + test_ft['day']  # 간단한 근사 값
test_ft['cos_day_of_year'] = np.cos(2 * np.pi * test_ft['day_of_year'] / 365)
test_ft['sin_day_of_year'] = np.sin(2 * np.pi * test_ft['day_of_year'] / 365)

# 요일에 대한 sin, cos 피처 추가
test_ft['cos_weekday'] = np.cos(2 * np.pi * test_ft['day_of_week'] / 7)  # 7일 기준으로 cos 값 추가
test_ft['sin_weekday'] = np.sin(2 * np.pi * test_ft['day_of_week'] / 7)  # 7일 기준으로 sin 값 추가

# 주차별 cos, sin 피처 추가
test_ft['cos_week'] = np.cos(2 * np.pi * test_ft['day_of_year'] / 52)  # 52주 기준으로 cos 값 추가
test_ft['sin_week'] = np.sin(2 * np.pi * test_ft['day_of_year'] / 52)  # 52주 기준으로 sin 값 추가

# 월에 대한 cos, sin 피처 추가
test_ft['cos_month'] = np.cos(2 * np.pi * test_ft['month'] / 12)  # 12달 기준으로 cos 값 추가
test_ft['sin_month'] = np.sin(2 * np.pi * test_ft['month'] / 12)  # 12달 기준으로 sin 값 추가


In [2322]:
train_ft['discomfort'] = 0.81 * train_ft['temp'] + 0.01 * train_ft['humidity'] * (0.99 * train_ft['temp'] - 14.3) + 46.3

train_ft['THI'] = 0.81 * train_ft['temp'] + 0.01 * train_ft['humidity'] * (0.99 * train_ft['temp'] - 14.3) + 46.3

# train_ft['solar_gen'] = train_ft['solar_rad'] * train_ft['sunshine'] * train_ft['solar_capa'] / 1000  # 태양광 발전량 - 단위: kW

# train_ft['final_solar_gen'] = train_ft['solar_gen'] * train_ft['ess_capa'] * train_ft['pcs_capa'] # PCS와 ESS를 고려한 최종 발전량 계산

train_ft['cooling_ratio'] = train_ft['cooling_area'] / train_ft['floor_area']

In [2323]:
test_ft['discomfort'] = 0.81 * test_ft['temp'] + 0.01 * test_ft['humidity'] * (0.99 * test_ft['temp'] - 14.3) + 46.3

test_ft['THI'] = 0.81 * test_ft['temp'] + 0.01 * test_ft['humidity'] * (0.99 * test_ft['temp'] - 14.3) + 46.3

# test_ft['solar_gen'] = test_ft['solar_rad'] * test_ft['sunshine'] * test_ft['solar_capa'] / 1000  # 태양광 발전량 - 단위: kW

# test_ft['final_solar_gen'] = test_ft['solar_gen'] * test_ft['ess_capa'] * test_ft['pcs_capa'] # PCS와 ESS를 고려한 최종 발전량 계산

test_ft['cooling_ratio'] = test_ft['cooling_area'] / test_ft['floor_area']


### 건물 번호별 df 정의

In [2324]:
#train_ft

# 1부터 36까지의 숫자에 따라 train_ft 데이터프레임을 생성하여 개별 변수로 저장
for i in range(1, 37):
    globals()[f'train_df_{i}'] = train_ft[train_ft['building_num'] == i]

In [2325]:
#test_ft

# 1부터 36까지의 숫자에 따라 train_ft 데이터프레임을 생성하여 개별 변수로 저장
for i in range(1, 37):
    globals()[f'test_df_{i}'] = test_ft[test_ft['building_num'] == i]

- df별로 모든 값이 결측치인 칼럼은 제거하기

In [2326]:
train_df_1.isnull().sum()

building_num          0
date_time             0
temp                  0
precip             1539
wind_ms               0
humidity              0
sunshine            767
solar_rad           767
energy_consum         0
building_type         0
floor_area            0
cooling_area          0
solar_capa         2040
ess_capa           2040
pcs_capa           2040
month                 0
day                   0
hour                  0
day_of_week           0
day_of_year           0
cos_hour              0
sin_hour              0
cos_day_of_year       0
sin_day_of_year       0
cos_weekday           0
sin_weekday           0
cos_week              0
sin_week              0
cos_month             0
sin_month             0
discomfort            0
THI                   0
cooling_ratio         0
dtype: int64

In [2327]:
def remove_nan_cols(dataframe):
    columns_to_keep = [col for col in dataframe.columns if not dataframe[col].isnull().all()]
    columns_to_drop = [col for col in dataframe.columns if dataframe[col].isnull().all()]
    return dataframe[columns_to_keep]

- df 별 칼럼 수, 제거된 칼럼 확인

In [2328]:
# train 데이터 처리
for i in range(1, 37):
    columns_to_drop = [col for col in globals()[f'train_df_{i}'].columns if globals()[f'train_df_{i}'][col].isnull().all()]
    globals()[f'train_df_{i}'] = remove_nan_cols(globals()[f'train_df_{i}'])
    columns_count = len(globals()[f'train_df_{i}'].columns)  # 현재 데이터프레임의 칼럼 수 계산

    print(f'{i}번 - {columns_count}, {columns_to_drop} 제거 ')


    for cols in columns_to_drop:
        if cols in globals()[f'test_df_{i}']:
             globals()[f'test_df_{i}'] = globals()[f'test_df_{i}'].drop(columns=cols, axis=1)



1번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
2번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
3번 - 31, ['ess_capa', 'pcs_capa'] 제거 
4번 - 31, ['ess_capa', 'pcs_capa'] 제거 
5번 - 32, ['solar_capa'] 제거 
6번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
7번 - 32, ['solar_capa'] 제거 
8번 - 29, ['solar_rad', 'solar_capa', 'ess_capa', 'pcs_capa'] 제거 
9번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
10번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
11번 - 30, ['solar_rad', 'ess_capa', 'pcs_capa'] 제거 
12번 - 31, ['ess_capa', 'pcs_capa'] 제거 
13번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
14번 - 31, ['ess_capa', 'pcs_capa'] 제거 
15번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
16번 - 31, ['ess_capa', 'pcs_capa'] 제거 
17번 - 30, ['solar_capa', 'ess_capa', 'pcs_capa'] 제거 
18번 - 31, ['ess_capa', 'pcs_capa'] 제거 
19번 - 32, ['solar_capa'] 제거 
20번 - 29, ['solar_rad', 'solar_capa', 'ess_capa', 'pcs_capa'] 제거 
21번 - 33, [] 제거 
22번 - 31, ['ess_capa', 'pcs_capa'] 제거 
23번 - 30, ['solar_capa',

In [2329]:
# #test 데이터

# for i in range(1, 37):
#     columns_to_drop = [col for col in globals()[f'test_df_{i}'].columns if globals()[f'test_df_{i}'][col].isnull().all()]
#     globals()[f'test_df_{i}'] = remove_nan_cols(globals()[f'test_df_{i}'])
#     columns_count = len(globals()[f'test_df_{i}'].columns)  # 현재 데이터프레임의 칼럼 수 계산
#     print(f'{i}번 - {columns_count} ')

## 건물 1

- 이상치 처리

In [2330]:
# train_df_1

In [2331]:
def replace_outliers_with_band(data):
    power_consumption = data['energy_consum'].values

    rolling_mean = np.convolve(power_consumption, np.ones(24)/24, mode='valid')
    power_consumption = power_consumption[23:]
    std_power = np.std(power_consumption)

    upper_band = rolling_mean + 1.8 * std_power
    lower_band = rolling_mean - 1.5 * std_power

    upper_outliers = np.where(power_consumption > upper_band)[0]
    lower_outliers = np.where(power_consumption < lower_band)[0]

    replaced_values = []  # 대체된 값들을 저장할 리스트

    # # 상한선 이상치 대체 및 정보 저장
    # for idx in upper_outliers:
    #     if power_consumption[idx] > upper_band[idx]:
    #         replaced_value = upper_band[idx]
    #         replaced_values.append((idx, power_consumption[idx], 'upper', replaced_value))
    #         data.loc[idx + 23, 'energy_consum'] = replaced_value

    # 하한선 이상치 대체 및 정보 저장
    for idx in lower_outliers:
        if power_consumption[idx] < lower_band[idx]:
            replaced_value = lower_band[idx]
            replaced_values.append((idx, power_consumption[idx], 'lower', replaced_value))
            data.loc[idx + 23, 'energy_consum'] = replaced_value

    return replaced_values

In [2332]:
replaced_values = replace_outliers_with_band(train_df_1)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_1.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2333]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad']

train_df_1.drop(columns=train_drop_cols,axis=1).columns

Index(['temp', 'precip', 'wind_ms', 'humidity', 'floor_area', 'cooling_area',
       'month', 'day', 'hour', 'day_of_week', 'day_of_year', 'cos_hour',
       'sin_hour', 'cos_day_of_year', 'sin_day_of_year', 'cos_weekday',
       'sin_weekday', 'cos_week', 'sin_week', 'cos_month', 'sin_month',
       'discomfort', 'THI', 'cooling_ratio'],
      dtype='object')

In [2334]:
test_drop_cols = ['building_num','date_time','building_type']

test_df_1.drop(columns=test_drop_cols,axis=1).columns

Index(['temp', 'precip', 'wind_ms', 'humidity', 'floor_area', 'cooling_area',
       'month', 'day', 'hour', 'day_of_week', 'day_of_year', 'cos_hour',
       'sin_hour', 'cos_day_of_year', 'sin_day_of_year', 'cos_weekday',
       'sin_weekday', 'cos_week', 'sin_week', 'cos_month', 'sin_month',
       'discomfort', 'THI', 'cooling_ratio'],
      dtype='object')

In [2335]:
train_x = train_df_1.drop(columns=train_drop_cols,axis=1)
train_y = train_df_1['energy_consum']

test_x = test_df_1.drop(columns=test_drop_cols,axis=1)

In [2336]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1800, 24), (240, 24), (1800,), (240,))

In [2337]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

#3.5400384169795878

[LightGBM] [Info] Total Bins 1341
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 21
[LightGBM] [Info] Start training from score 2622.530972
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[38]	training's l2: 15819.4	valid_1's l2: 48119.1


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2338]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2339]:
_smape(y_valid, x_pred)

3.597742572413208

In [2340]:
model.fit(train_x, train_y)
y_pred = model.predict(test_x)
y_pred.shape

[LightGBM] [Info] Total Bins 1388
[LightGBM] [Info] Number of data points in the train set: 2040, number of used features: 21
[LightGBM] [Info] Start training from score 2653.789681


(168,)

## 건물 2

- 이상치 처리

In [2341]:
replaced_values = replace_outliers_with_band(train_df_2)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_2.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2342]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2343]:
train_x = train_df_2.drop(columns=train_drop_cols,axis=1)
train_y = train_df_2['energy_consum']

test_x = test_df_2.drop(columns=test_drop_cols,axis=1)

In [2344]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1825, 23), (240, 23), (1825,), (240,))

In [2345]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1300
[LightGBM] [Info] Number of data points in the train set: 1825, number of used features: 20
[LightGBM] [Info] Start training from score 1671.521129
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[14]	training's l2: 43537.1	valid_1's l2: 54603.2


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2346]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2347]:
_smape(y_valid, x_pred)

5.724325431753672

In [2348]:
model.fit(train_x, train_y)
pred = np.concatenate((y_pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1363
[LightGBM] [Info] Number of data points in the train set: 2065, number of used features: 23
[LightGBM] [Info] Start training from score 1668.795957


(336,)

## 건물 3

In [2349]:
test_ft.fillna(0)

,building_num,date_time,temp,precip,wind_ms,humidity,building_type,floor_area,cooling_area,solar_capa,...,sin_day_of_year,cos_weekday,sin_weekday,cos_week,sin_week,cos_month,sin_month,discomfort,THI,cooling_ratio
0,1,2022-06-01 00:00:00,23.5,0.0,2.2,72,건물기타,110634.00,39570.00,0.0,...,0.025818,-0.222521,0.974928,-0.992709,0.120537,-1.0,1.224647e-16,71.78980,71.78980,0.357666
1,1,2022-06-01 01:00:00,23.0,0.0,0.9,72,건물기타,110634.00,39570.00,0.0,...,0.025818,-0.222521,0.974928,-0.992709,0.120537,-1.0,1.224647e-16,71.02840,71.02840,0.357666
2,1,2022-06-01 02:00:00,22.7,0.0,1.5,75,건물기타,110634.00,39570.00,0.0,...,0.025818,-0.222521,0.974928,-0.992709,0.120537,-1.0,1.224647e-16,70.81675,70.81675,0.357666
3,1,2022-06-01 03:00:00,22.1,0.0,1.3,78,건물기타,110634.00,39570.00,0.0,...,0.025818,-0.222521,0.974928,-0.992709,0.120537,-1.0,1.224647e-16,70.11262,70.11262,0.357666
4,1,2022-06-01 04:00:00,21.8,0.0,1.0,77,건물기타,110634.00,39570.00,0.0,...,0.025818,-0.222521,0.974928,-0.992709,0.120537,-1.0,1.224647e-16,69.56514,69.56514,0.357666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,2022-06-20 19:00:00,22.5,0.0,0.9,84,호텔및리조트,57497.84,40035.23,0.0,...,-0.296713,1.000000,0.000000,0.568065,-0.822984,-1.0,1.224647e-16,71.22400,71.22400,0.696291
16796,100,2022-06-20 20:00:00,20.7,0.0,0.4,95,호텔및리조트,57497.84,40035.23,0.0,...,-0.296713,1.000000,0.000000,0.568065,-0.822984,-1.0,1.224647e-16,68.95035,68.95035,0.696291
16797,100,2022-06-20 21:00:00,20.2,0.0,0.4,98,호텔및리조트,57497.84,40035.23,0.0,...,-0.296713,1.000000,0.000000,0.568065,-0.822984,-1.0,1.224647e-16,68.24604,68.24604,0.696291
16798,100,2022-06-20 22:00:00,20.1,0.0,1.1,97,호텔및리조트,57497.84,40035.23,0.0,...,-0.296713,1.000000,0.000000,0.568065,-0.822984,-1.0,1.224647e-16,68.01203,68.01203,0.696291


- 이상치 처리

In [2350]:
replaced_values = replace_outliers_with_band(train_df_3)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_3.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2351]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2352]:
train_x = train_df_3.drop(columns=train_drop_cols,axis=1)
train_y = train_df_3['energy_consum']

test_x = test_df_3.drop(columns=test_drop_cols,axis=1)

In [2353]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1820, 24), (240, 24), (1820,), (240,))

In [2354]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1297
[LightGBM] [Info] Number of data points in the train set: 1820, number of used features: 20
[LightGBM] [Info] Start training from score 1449.276825
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[13]	training's l2: 59906.5	valid_1's l2: 58383.8


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2355]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2356]:
_smape(y_valid, x_pred)

7.810996807489477

In [2357]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1365
[LightGBM] [Info] Number of data points in the train set: 2060, number of used features: 24
[LightGBM] [Info] Start training from score 1439.967379


(504,)

## 건물 4

- 이상치 처리

In [2358]:
replaced_values = replace_outliers_with_band(train_df_4)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_4.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2359]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2360]:
train_x = train_df_4.drop(columns=train_drop_cols,axis=1)
train_y = train_df_4['energy_consum']

test_x = test_df_4.drop(columns=test_drop_cols,axis=1)

In [2361]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1805, 24), (240, 24), (1805,), (240,))

In [2362]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1293
[LightGBM] [Info] Number of data points in the train set: 1805, number of used features: 20
[LightGBM] [Info] Start training from score 988.292693
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[37]	training's l2: 1112.86	valid_1's l2: 5194.09


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2363]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2364]:
_smape(y_valid, x_pred)

3.0704854834744273

In [2365]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1353
[LightGBM] [Info] Number of data points in the train set: 2045, number of used features: 20
[LightGBM] [Info] Start training from score 994.536378


(672,)

## 건물 5

- 이상치 처리

In [2366]:
replaced_values = replace_outliers_with_band(train_df_5)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_5.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2367]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2368]:
train_x = train_df_5.drop(columns=train_drop_cols,axis=1)
train_y = train_df_5['energy_consum']

test_x = test_df_5.drop(columns=test_drop_cols,axis=1)

In [2369]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1804, 25), (240, 25), (1804,), (240,))

In [2370]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1293
[LightGBM] [Info] Number of data points in the train set: 1804, number of used features: 20
[LightGBM] [Info] Start training from score 2744.252259
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[38]	training's l2: 52422.3	valid_1's l2: 105312


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2371]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2372]:
_smape(y_valid, x_pred)

3.9507213456830073

In [2373]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1353
[LightGBM] [Info] Number of data points in the train set: 2044, number of used features: 20
[LightGBM] [Info] Start training from score 2737.262359


(840,)

## 건물 6

- 이상치 처리

In [2374]:
replaced_values = replace_outliers_with_band(train_df_6)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_6.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2375]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2376]:
train_x = train_df_6.drop(columns=train_drop_cols,axis=1)
train_y = train_df_6['energy_consum']

test_x = test_df_6.drop(columns=test_drop_cols,axis=1)

In [2377]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1803, 23), (240, 23), (1803,), (240,))

In [2378]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1304
[LightGBM] [Info] Number of data points in the train set: 1803, number of used features: 20
[LightGBM] [Info] Start training from score 1956.962663
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[48]	training's l2: 3055.79	valid_1's l2: 20159.4


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2379]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2380]:
_smape(y_valid, x_pred)

2.156229676330258

In [2381]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1362
[LightGBM] [Info] Number of data points in the train set: 2043, number of used features: 20
[LightGBM] [Info] Start training from score 1971.644278


(1008,)

## 건물 7

- 이상치 처리

In [2382]:
replaced_values = replace_outliers_with_band(train_df_7)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_7.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2383]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2384]:
train_x = train_df_7.drop(columns=train_drop_cols,axis=1)
train_y = train_df_7['energy_consum']

test_x = test_df_7.drop(columns=test_drop_cols,axis=1)

In [2385]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1864, 25), (240, 25), (1864,), (240,))

In [2386]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1315
[LightGBM] [Info] Number of data points in the train set: 1864, number of used features: 20
[LightGBM] [Info] Start training from score 780.322790
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[64]	training's l2: 1259.79	valid_1's l2: 48614.8


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2387]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2388]:
_smape(y_valid, x_pred)

12.196554227587008

In [2389]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1377
[LightGBM] [Info] Number of data points in the train set: 2104, number of used features: 25
[LightGBM] [Info] Start training from score 760.941873


(1176,)

## 건물 8

- 이상치 처리

In [2390]:
replaced_values = replace_outliers_with_band(train_df_8)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_8.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2391]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','precip'] #solar rad 없음
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2392]:
train_df_8.columns

Index(['building_num', 'date_time', 'temp', 'precip', 'wind_ms', 'humidity',
       'sunshine', 'energy_consum', 'building_type', 'floor_area',
       'cooling_area', 'month', 'day', 'hour', 'day_of_week', 'day_of_year',
       'cos_hour', 'sin_hour', 'cos_day_of_year', 'sin_day_of_year',
       'cos_weekday', 'sin_weekday', 'cos_week', 'sin_week', 'cos_month',
       'sin_month', 'discomfort', 'THI', 'cooling_ratio'],
      dtype='object')

In [2393]:
train_x = train_df_8.drop(columns=train_drop_cols,axis=1)
train_y = train_df_8['energy_consum']

test_x = test_df_8.drop(columns=test_drop_cols,axis=1)

In [2394]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1891, 23), (240, 23), (1891,), (240,))

In [2395]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1305
[LightGBM] [Info] Number of data points in the train set: 1891, number of used features: 20
[LightGBM] [Info] Start training from score 1326.023607
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[77]	training's l2: 1531.21	valid_1's l2: 12246.8


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2396]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2397]:
_smape(y_valid, x_pred)

3.6084711952828386

In [2398]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1359
[LightGBM] [Info] Number of data points in the train set: 2131, number of used features: 23
[LightGBM] [Info] Start training from score 1317.380234


(1344,)

## 건물 9

- 이상치 처리

In [2399]:
replaced_values = replace_outliers_with_band(train_df_9)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_9.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2400]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2401]:
train_x = train_df_9.drop(columns=train_drop_cols,axis=1)
train_y = train_df_9['energy_consum']

test_x = test_df_9.drop(columns=test_drop_cols,axis=1)

In [2402]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1852, 23), (240, 23), (1852,), (240,))

In [2403]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1320
[LightGBM] [Info] Number of data points in the train set: 1852, number of used features: 20
[LightGBM] [Info] Start training from score 2324.360864
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[38]	training's l2: 4868.82	valid_1's l2: 13752.4


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2404]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2405]:
_smape(y_valid, x_pred)

2.5163777405452716

In [2406]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1375
[LightGBM] [Info] Number of data points in the train set: 2092, number of used features: 23
[LightGBM] [Info] Start training from score 2314.820139


(1512,)

## 건물 10

- 이상치 처리

In [2407]:
replaced_values = replace_outliers_with_band(train_df_10)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_2.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2408]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2409]:
train_x = train_df_10.drop(columns=train_drop_cols,axis=1)
train_y = train_df_10['energy_consum']

test_x = test_df_10.drop(columns=test_drop_cols,axis=1)

In [2410]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1816, 23), (240, 23), (1816,), (240,))

In [2411]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1312
[LightGBM] [Info] Number of data points in the train set: 1816, number of used features: 20
[LightGBM] [Info] Start training from score 4048.277049
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[29]	training's l2: 20700.2	valid_1's l2: 127115


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2412]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2413]:
_smape(y_valid, x_pred)

3.5958674686294776

In [2414]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1371
[LightGBM] [Info] Number of data points in the train set: 2056, number of used features: 20
[LightGBM] [Info] Start training from score 4054.513458


(1680,)

## 건물 11

- 이상치 처리

In [2415]:
replaced_values = replace_outliers_with_band(train_df_11)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_11.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2416]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','precip'] #solar_rad
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2417]:
train_x = train_df_11.drop(columns=train_drop_cols,axis=1)
train_y = train_df_11['energy_consum']

test_x = test_df_11.drop(columns=test_drop_cols,axis=1)

In [2418]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1805, 24), (240, 24), (1805,), (240,))

In [2419]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1293
[LightGBM] [Info] Number of data points in the train set: 1805, number of used features: 20
[LightGBM] [Info] Start training from score 1927.580941
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[74]	training's l2: 1973.97	valid_1's l2: 11057.3


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2420]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2421]:
_smape(y_valid, x_pred)

1.8777286024350508

In [2422]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1350
[LightGBM] [Info] Number of data points in the train set: 2045, number of used features: 20
[LightGBM] [Info] Start training from score 1929.349082


(1848,)

## 건물 12

- 이상치 처리

In [2423]:
replaced_values = replace_outliers_with_band(train_df_12)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_12.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2424]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2425]:
train_x = train_df_12.drop(columns=train_drop_cols,axis=1)
train_y = train_df_12['energy_consum']

test_x = test_df_12.drop(columns=test_drop_cols,axis=1)

In [2426]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1833, 24), (240, 24), (1833,), (240,))

In [2427]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1311
[LightGBM] [Info] Number of data points in the train set: 1833, number of used features: 20
[LightGBM] [Info] Start training from score 1432.420850
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[37]	training's l2: 1817.54	valid_1's l2: 8162.19


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2428]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2429]:
_smape(y_valid, x_pred)

2.1516533762097483

In [2430]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1374
[LightGBM] [Info] Number of data points in the train set: 2073, number of used features: 24
[LightGBM] [Info] Start training from score 1438.157744


(2016,)

## 건물 13

- 이상치 처리

In [2431]:
replaced_values = replace_outliers_with_band(train_df_13)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_13.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2432]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2433]:
train_x = train_df_13.drop(columns=train_drop_cols,axis=1)
train_y = train_df_13['energy_consum']

test_x = test_df_13.drop(columns=test_drop_cols,axis=1)

In [2434]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1840, 23), (240, 23), (1840,), (240,))

In [2435]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1317
[LightGBM] [Info] Number of data points in the train set: 1840, number of used features: 20
[LightGBM] [Info] Start training from score 2529.911348
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[15]	training's l2: 25781	valid_1's l2: 32191.9


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2436]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2437]:
_smape(y_valid, x_pred)

2.9772368830641605

In [2438]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1380
[LightGBM] [Info] Number of data points in the train set: 2080, number of used features: 23
[LightGBM] [Info] Start training from score 2526.586211


(2184,)

## 건물 14

- 이상치 처리

In [2439]:
replaced_values = replace_outliers_with_band(train_df_14)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_14.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2440]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2441]:
train_x = train_df_14.drop(columns=train_drop_cols,axis=1)
train_y = train_df_14['energy_consum']

test_x = test_df_14.drop(columns=test_drop_cols,axis=1)

In [2442]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1904, 24), (240, 24), (1904,), (240,))

In [2443]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1309
[LightGBM] [Info] Number of data points in the train set: 1904, number of used features: 20
[LightGBM] [Info] Start training from score 1916.145629
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[40]	training's l2: 35926	valid_1's l2: 124441


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2444]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2445]:
_smape(y_valid, x_pred)

9.152098810809312

In [2446]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1362
[LightGBM] [Info] Number of data points in the train set: 2144, number of used features: 24
[LightGBM] [Info] Start training from score 1881.260710


(2352,)

## 건물 15

- 이상치 처리

In [2447]:
replaced_values = replace_outliers_with_band(train_df_2)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_2.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2448]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2449]:
train_x = train_df_2.drop(columns=train_drop_cols,axis=1)
train_y = train_df_2['energy_consum']

test_x = test_df_2.drop(columns=test_drop_cols,axis=1)

In [2450]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1839, 23), (240, 23), (1839,), (240,))

In [2451]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1302
[LightGBM] [Info] Number of data points in the train set: 1839, number of used features: 20
[LightGBM] [Info] Start training from score 1671.782741
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[8]	training's l2: 97214.1	valid_1's l2: 400804


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2452]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2453]:
_smape(y_valid, x_pred)

10.27823663579252

In [2454]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1363
[LightGBM] [Info] Number of data points in the train set: 2079, number of used features: 23
[LightGBM] [Info] Start training from score 1682.451524


(2520,)

## 건물 16

- 이상치 처리

In [2455]:
replaced_values = replace_outliers_with_band(train_df_16)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_16.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2456]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2457]:
train_x = train_df_16.drop(columns=train_drop_cols,axis=1)
train_y = train_df_16['energy_consum']

test_x = test_df_16.drop(columns=test_drop_cols,axis=1)

In [2458]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((2069, 24), (240, 24), (2069,), (240,))

In [2459]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1351
[LightGBM] [Info] Number of data points in the train set: 2069, number of used features: 24
[LightGBM] [Info] Start training from score 3376.703843
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[23]	training's l2: 41754.5	valid_1's l2: 21273.6


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2460]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2461]:
_smape(y_valid, x_pred)

4.155723661471707

In [2462]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1351
[LightGBM] [Info] Number of data points in the train set: 2309, number of used features: 24
[LightGBM] [Info] Start training from score 3185.959034


(2688,)

## 건물 17

- 이상치 처리

In [2463]:
replaced_values = replace_outliers_with_band(train_df_17)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_17.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2464]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2465]:
train_x = train_df_17.drop(columns=train_drop_cols,axis=1)
train_y = train_df_17['energy_consum']

test_x = test_df_17.drop(columns=test_drop_cols,axis=1)

In [2466]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1828, 23), (240, 23), (1828,), (240,))

In [2467]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1301
[LightGBM] [Info] Number of data points in the train set: 1828, number of used features: 20
[LightGBM] [Info] Start training from score 966.001756
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[22]	training's l2: 6168.63	valid_1's l2: 50751.8


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2468]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2469]:
_smape(y_valid, x_pred)

7.323938536043037

In [2470]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1362
[LightGBM] [Info] Number of data points in the train set: 2068, number of used features: 23
[LightGBM] [Info] Start training from score 971.891331


(2856,)

## 건물 18

- 이상치 처리

In [2471]:
replaced_values = replace_outliers_with_band(train_df_18)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_18.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2472]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2473]:
train_x = train_df_18.drop(columns=train_drop_cols,axis=1)
train_y = train_df_18['energy_consum']

test_x = test_df_18.drop(columns=test_drop_cols,axis=1)

In [2474]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1808, 24), (240, 24), (1808,), (240,))

In [2475]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1304
[LightGBM] [Info] Number of data points in the train set: 1808, number of used features: 20
[LightGBM] [Info] Start training from score 1436.611643
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[24]	training's l2: 7280.37	valid_1's l2: 48702.9


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2476]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2477]:
_smape(y_valid, x_pred)

4.276641306583906

In [2478]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1362
[LightGBM] [Info] Number of data points in the train set: 2048, number of used features: 20
[LightGBM] [Info] Start training from score 1451.678989


(3024,)

## 건물 19

- 이상치 처리

In [2479]:
replaced_values = replace_outliers_with_band(train_df_19)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_19.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2480]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2481]:
train_x = train_df_19.drop(columns=train_drop_cols,axis=1)
train_y = train_df_19['energy_consum']

test_x = test_df_19.drop(columns=test_drop_cols,axis=1)

In [2482]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1813, 25), (240, 25), (1813,), (240,))

In [2483]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1294
[LightGBM] [Info] Number of data points in the train set: 1813, number of used features: 20
[LightGBM] [Info] Start training from score 1408.483559
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[22]	training's l2: 12243.6	valid_1's l2: 68226.9


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2484]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2485]:
_smape(y_valid, x_pred)

6.724716697470674

In [2486]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1353
[LightGBM] [Info] Number of data points in the train set: 2053, number of used features: 20
[LightGBM] [Info] Start training from score 1419.024466


(3192,)

## 건물 20

- 이상치 처리

In [2487]:
replaced_values = replace_outliers_with_band(train_df_20)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_20.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2488]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','precip'] #solar_rad
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2489]:
train_x = train_df_20.drop(columns=train_drop_cols,axis=1)
train_y = train_df_20['energy_consum']

test_x = test_df_20.drop(columns=test_drop_cols,axis=1)

In [2490]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1800, 23), (240, 23), (1800,), (240,))

In [2491]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1285
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 20
[LightGBM] [Info] Start training from score 1708.765068
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[22]	training's l2: 18994	valid_1's l2: 254940


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2492]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2493]:
_smape(y_valid, x_pred)

6.953786391103097

In [2494]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1326
[LightGBM] [Info] Number of data points in the train set: 2040, number of used features: 20
[LightGBM] [Info] Start training from score 1722.787530


(3360,)

## 건물 21

- 이상치 처리

In [2495]:
replaced_values = replace_outliers_with_band(train_df_21)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_21.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2496]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2497]:
train_x = train_df_21.drop(columns=train_drop_cols,axis=1)
train_y = train_df_21['energy_consum']

test_x = test_df_21.drop(columns=test_drop_cols,axis=1)

In [2498]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1801, 26), (240, 26), (1801,), (240,))

In [2499]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1283
[LightGBM] [Info] Number of data points in the train set: 1801, number of used features: 20
[LightGBM] [Info] Start training from score 1973.311429
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[22]	training's l2: 19137.9	valid_1's l2: 95941.7


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2500]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2501]:
_smape(y_valid, x_pred)

5.051542080490981

In [2502]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1340
[LightGBM] [Info] Number of data points in the train set: 2041, number of used features: 20
[LightGBM] [Info] Start training from score 1988.770845


(3528,)

## 건물 22

- 이상치 처리

In [2503]:
replaced_values = replace_outliers_with_band(train_df_22)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_22.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2504]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2505]:
train_x = train_df_22.drop(columns=train_drop_cols,axis=1)
train_y = train_df_22['energy_consum']

test_x = test_df_22.drop(columns=test_drop_cols,axis=1)

In [2506]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1822, 24), (240, 24), (1822,), (240,))

In [2507]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1279
[LightGBM] [Info] Number of data points in the train set: 1822, number of used features: 20
[LightGBM] [Info] Start training from score 1230.799496
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[37]	training's l2: 1812.38	valid_1's l2: 16178.9


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2508]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2509]:
_smape(y_valid, x_pred)

3.524159298890983

In [2510]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1348
[LightGBM] [Info] Number of data points in the train set: 2062, number of used features: 24
[LightGBM] [Info] Start training from score 1232.508255


(3696,)

## 건물 23

- 이상치 처리

In [2511]:
replaced_values = replace_outliers_with_band(train_df_23)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_23.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2512]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2513]:
train_x = train_df_23.drop(columns=train_drop_cols,axis=1)
train_y = train_df_23['energy_consum']

test_x = test_df_23.drop(columns=test_drop_cols,axis=1)

In [2514]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1804, 23), (240, 23), (1804,), (240,))

In [2515]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1276
[LightGBM] [Info] Number of data points in the train set: 1804, number of used features: 20
[LightGBM] [Info] Start training from score 1259.223393
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[24]	training's l2: 1225.03	valid_1's l2: 17194.5


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2516]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2517]:
_smape(y_valid, x_pred)

2.5118269370371262

In [2518]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1335
[LightGBM] [Info] Number of data points in the train set: 2044, number of used features: 20
[LightGBM] [Info] Start training from score 1263.823594


(3864,)

## 건물 24

- 이상치 처리

In [2519]:
replaced_values = replace_outliers_with_band(train_df_24)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_24.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2520]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2521]:
train_x = train_df_24.drop(columns=train_drop_cols,axis=1)
train_y = train_df_24['energy_consum']

test_x = test_df_24.drop(columns=test_drop_cols,axis=1)

In [2522]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1802, 24), (240, 24), (1802,), (240,))

In [2523]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 1802, number of used features: 20
[LightGBM] [Info] Start training from score 9993.332965
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[20]	training's l2: 233672	valid_1's l2: 1.43432e+06


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2524]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2525]:
_smape(y_valid, x_pred)

2.912555745280679

In [2526]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1353
[LightGBM] [Info] Number of data points in the train set: 2042, number of used features: 20
[LightGBM] [Info] Start training from score 10041.156217


(4032,)

## 건물 25

- 이상치 처리

In [2527]:
replaced_values = replace_outliers_with_band(train_df_25)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_25.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2528]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2529]:
train_x = train_df_25.drop(columns=train_drop_cols,axis=1)
train_y = train_df_25['energy_consum']

test_x = test_df_25.drop(columns=test_drop_cols,axis=1)

In [2530]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1800, 24), (240, 24), (1800,), (240,))

In [2531]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1289
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 20
[LightGBM] [Info] Start training from score 7229.745061
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[23]	training's l2: 106285	valid_1's l2: 1.01396e+06


/usr/local/lib/python3.10/dist-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2532]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2533]:
_smape(y_valid, x_pred)

3.098230579975161

In [2534]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1335
[LightGBM] [Info] Number of data points in the train set: 2040, number of used features: 20
[LightGBM] [Info] Start training from score 7264.323584


(4200,)

## 건물 26

- 이상치 처리

In [2535]:
replaced_values = replace_outliers_with_band(train_df_26)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_26.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2536]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2537]:
train_x = train_df_26.drop(columns=train_drop_cols,axis=1)
train_y = train_df_26['energy_consum']

test_x = test_df_26.drop(columns=test_drop_cols,axis=1)

In [2538]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1800, 24), (240, 24), (1800,), (240,))

In [2539]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1297
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 20
[LightGBM] [Info] Start training from score 3518.006933
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[22]	training's l2: 26985.4	valid_1's l2: 155628


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2540]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2541]:
_smape(y_valid, x_pred)

2.649225475215552

In [2542]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1339
[LightGBM] [Info] Number of data points in the train set: 2040, number of used features: 20
[LightGBM] [Info] Start training from score 3529.790117


(4368,)

## 건물 27

- 이상치 처리

In [2543]:
replaced_values = replace_outliers_with_band(train_df_27)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_27.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2544]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2545]:
train_x = train_df_27.drop(columns=train_drop_cols,axis=1)
train_y = train_df_27['energy_consum']

test_x = test_df_27.drop(columns=test_drop_cols,axis=1)

In [2546]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1800, 24), (240, 24), (1800,), (240,))

In [2547]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1298
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 20
[LightGBM] [Info] Start training from score 17397.379988
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[17]	training's l2: 540658	valid_1's l2: 1.63696e+06


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2548]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2549]:
_smape(y_valid, x_pred)

1.9544426929689738

In [2550]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1339
[LightGBM] [Info] Number of data points in the train set: 2040, number of used features: 20
[LightGBM] [Info] Start training from score 17453.642143


(4536,)

## 건물 28

- 이상치 처리

In [2551]:
replaced_values = replace_outliers_with_band(train_df_28)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_28.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2552]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','precip'] #solar_rad
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2553]:
train_x = train_df_28.drop(columns=train_drop_cols,axis=1)
train_y = train_df_28['energy_consum']

test_x = test_df_28.drop(columns=test_drop_cols,axis=1)

In [2554]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1810, 24), (240, 24), (1810,), (240,))

In [2555]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 1810, number of used features: 20
[LightGBM] [Info] Start training from score 1734.283094
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[21]	training's l2: 18093.2	valid_1's l2: 48154.3


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2556]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2557]:
_smape(y_valid, x_pred)

3.79769854312523

In [2558]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1350
[LightGBM] [Info] Number of data points in the train set: 2050, number of used features: 20
[LightGBM] [Info] Start training from score 1728.555457


(4704,)

## 건물 29

- 이상치 처리

In [2559]:
replaced_values = replace_outliers_with_band(train_df_29)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_29.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2560]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','precip'] #solar_rad
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2561]:
train_x = train_df_29.drop(columns=train_drop_cols,axis=1)
train_y = train_df_29['energy_consum']

test_x = test_df_29.drop(columns=test_drop_cols,axis=1)

In [2562]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1808, 24), (240, 24), (1808,), (240,))

In [2563]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1292
[LightGBM] [Info] Number of data points in the train set: 1808, number of used features: 20
[LightGBM] [Info] Start training from score 1448.236311
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[13]	training's l2: 16755.6	valid_1's l2: 44423.9


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2564]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2565]:
_smape(y_valid, x_pred)

4.312418138865363

In [2566]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1348
[LightGBM] [Info] Number of data points in the train set: 2048, number of used features: 20
[LightGBM] [Info] Start training from score 1448.941765


(4872,)

## 건물 30

- 이상치 처리

In [2567]:
replaced_values = replace_outliers_with_band(train_df_30)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_30.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2568]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2569]:
train_x = train_df_30.drop(columns=train_drop_cols,axis=1)
train_y = train_df_30['energy_consum']

test_x = test_df_30.drop(columns=test_drop_cols,axis=1)

In [2570]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1840, 24), (240, 24), (1840,), (240,))

In [2571]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1296
[LightGBM] [Info] Number of data points in the train set: 1840, number of used features: 20
[LightGBM] [Info] Start training from score 837.597799
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[100]	training's l2: 986.527	valid_1's l2: 25217.6


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2572]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2573]:
_smape(y_valid, x_pred)

7.513455731648783

In [2574]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1359
[LightGBM] [Info] Number of data points in the train set: 2080, number of used features: 24
[LightGBM] [Info] Start training from score 833.614237


(5040,)

## 건물 31

- 이상치 처리

In [2575]:
replaced_values = replace_outliers_with_band(train_df_31)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_31.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2576]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2577]:
train_x = train_df_31.drop(columns=train_drop_cols,axis=1)
train_y = train_df_31['energy_consum']

test_x = test_df_31.drop(columns=test_drop_cols,axis=1)

In [2578]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1812, 24), (240, 24), (1812,), (240,))

In [2579]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1298
[LightGBM] [Info] Number of data points in the train set: 1812, number of used features: 20
[LightGBM] [Info] Start training from score 1355.648045
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[38]	training's l2: 2515.07	valid_1's l2: 21017.3


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2580]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2581]:
_smape(y_valid, x_pred)

3.3203106064483565

In [2582]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1358
[LightGBM] [Info] Number of data points in the train set: 2052, number of used features: 20
[LightGBM] [Info] Start training from score 1353.470236


(5208,)

## 건물 32

- 이상치 처리

In [2583]:
replaced_values = replace_outliers_with_band(train_df_32)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_32.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2584]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2585]:
train_x = train_df_32.drop(columns=train_drop_cols,axis=1)
train_y = train_df_32['energy_consum']

test_x = test_df_32.drop(columns=test_drop_cols,axis=1)

In [2586]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1800, 23), (240, 23), (1800,), (240,))

In [2587]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1290
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 20
[LightGBM] [Info] Start training from score 9861.091387
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[68]	training's l2: 354.865	valid_1's l2: 2978.88


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2588]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2589]:
_smape(y_valid, x_pred)

0.21712358857586211

In [2590]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1336
[LightGBM] [Info] Number of data points in the train set: 2040, number of used features: 20
[LightGBM] [Info] Start training from score 9865.783576


(5376,)

## 건물 33

- 이상치 처리

In [2591]:
replaced_values = replace_outliers_with_band(train_df_33)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_33.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2592]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2593]:
train_x = train_df_33.drop(columns=train_drop_cols,axis=1)
train_y = train_df_33['energy_consum']

test_x = test_df_33.drop(columns=test_drop_cols,axis=1)

In [2594]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1800, 23), (240, 23), (1800,), (240,))

In [2595]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1289
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 20
[LightGBM] [Info] Start training from score 8689.485592
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[100]	training's l2: 195.509	valid_1's l2: 1612.6


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2596]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2597]:
_smape(y_valid, x_pred)

0.18471761663300343

In [2598]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1335
[LightGBM] [Info] Number of data points in the train set: 2040, number of used features: 20
[LightGBM] [Info] Start training from score 8700.811057


(5544,)

## 건물 34

- 이상치 처리

In [2599]:
replaced_values = replace_outliers_with_band(train_df_34)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_34.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2600]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2601]:
train_x = train_df_34.drop(columns=train_drop_cols,axis=1)
train_y = train_df_34['energy_consum']

test_x = test_df_34.drop(columns=test_drop_cols,axis=1)

In [2602]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1801, 23), (240, 23), (1801,), (240,))

In [2603]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 1801, number of used features: 20
[LightGBM] [Info] Start training from score 3348.646440
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[57]	training's l2: 362.917	valid_1's l2: 1013.89


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2604]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2605]:
_smape(y_valid, x_pred)

0.34404951362998276

In [2606]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1353
[LightGBM] [Info] Number of data points in the train set: 2041, number of used features: 20
[LightGBM] [Info] Start training from score 3352.664958


(5712,)

## 건물 35

- 이상치 처리

In [2607]:
replaced_values = replace_outliers_with_band(train_df_35)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_35.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2608]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2609]:
train_x = train_df_35.drop(columns=train_drop_cols,axis=1)
train_y = train_df_35['energy_consum']

test_x = test_df_35.drop(columns=test_drop_cols,axis=1)

In [2610]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1805, 23), (240, 23), (1805,), (240,))

In [2611]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1293
[LightGBM] [Info] Number of data points in the train set: 1805, number of used features: 20
[LightGBM] [Info] Start training from score 2170.705264
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[27]	training's l2: 75.9475	valid_1's l2: 247.347


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2612]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2613]:
_smape(y_valid, x_pred)

0.27707922048394196

In [2614]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1353
[LightGBM] [Info] Number of data points in the train set: 2045, number of used features: 20
[LightGBM] [Info] Start training from score 2172.250901


(5880,)

## 건물 36

- 이상치 처리

In [2615]:
replaced_values = replace_outliers_with_band(train_df_36)

for idx, original_value, band_type, replaced_value in replaced_values:
    train_df_36.loc[idx + 23, 'energy_consum'] = replaced_value

- 칼럼 drop

In [2616]:
train_drop_cols = ['energy_consum','building_num','date_time','building_type','sunshine','solar_rad','precip']
test_drop_cols = ['building_num','date_time','building_type','precip']

In [2617]:
train_x = train_df_36.drop(columns=train_drop_cols,axis=1)
train_y = train_df_36['energy_consum']

test_x = test_df_36.drop(columns=test_drop_cols,axis=1)

In [2618]:
# 마지막 10일을 검증셋으로 설정
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = train_y, X = train_x, test_size = 240)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1811, 23), (240, 23), (1811,), (240,))

In [2619]:
model = LGBMRegressor(random_state=SEED, objective='regression', force_row_wise=True)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], callbacks = [early_stopping(300)])

[LightGBM] [Info] Total Bins 1304
[LightGBM] [Info] Number of data points in the train set: 1811, number of used features: 20
[LightGBM] [Info] Start training from score 2891.990391
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[86]	training's l2: 44.0207	valid_1's l2: 768.1


LGBMRegressor(force_row_wise=True, objective='regression', random_state=42)

In [2620]:
x_pred = model.predict(x_valid)
x_pred = pd.Series(x_pred)
# x_pred.index = np.arange(y_valid.index[0], y_valid.index[-1]+1)
# plot_series(y_train, y_valid, pd.Series(x_pred), markers=[',' , ',', ','])

In [2621]:
_smape(y_valid, x_pred)

0.3740976985043156

In [2622]:
model.fit(train_x, train_y)
pred = np.concatenate((pred, model.predict(test_x)))
pred.shape

[LightGBM] [Info] Total Bins 1362
[LightGBM] [Info] Number of data points in the train set: 2051, number of used features: 20
[LightGBM] [Info] Start training from score 2892.629258


(6048,)

In [2623]:
test_ft.shape

(16800, 30)

In [2624]:
mask = test['건물번호'] == 36
test[mask]

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
5880,36_20220825 00,36,20220825 00,22.8,0.0,1.1,75
5881,36_20220825 01,36,20220825 01,22.6,0.0,1.0,75
5882,36_20220825 02,36,20220825 02,22.2,0.0,1.0,77
5883,36_20220825 03,36,20220825 03,21.4,0.0,1.0,83
5884,36_20220825 04,36,20220825 04,21.2,0.0,0.3,85
...,...,...,...,...,...,...,...
6043,36_20220831 19,36,20220831 19,22.4,0.0,1.1,84
6044,36_20220831 20,36,20220831 20,21.5,0.0,0.8,86
6045,36_20220831 21,36,20220831 21,21.8,0.0,0.9,86
6046,36_20220831 22,36,20220831 22,21.1,0.0,0.4,93


In [2625]:
submission.iloc[:6048]

,num_date_time,answer
0,1_20220825 00,0
1,1_20220825 01,0
2,1_20220825 02,0
3,1_20220825 03,0
4,1_20220825 04,0
...,...,...
6043,36_20220831 19,0
6044,36_20220831 20,0
6045,36_20220831 21,0
6046,36_20220831 22,0


In [2626]:
len(pred)

6048

In [2627]:
submission.iloc[:6048,1] = pred

In [2628]:
submission.iloc[:6048]

,num_date_time,answer
0,1_20220825 00,1051.134133
1,1_20220825 01,1014.054625
2,1_20220825 02,1010.312740
3,1_20220825 03,1003.938728
4,1_20220825 04,997.774250
...,...,...
6043,36_20220831 19,2878.042448
6044,36_20220831 20,2882.184163
6045,36_20220831 21,2879.621748
6046,36_20220831 22,2870.958227


In [2629]:
submission.to_csv('submission_1_36.csv', index=False)